In [1]:
import pandas as pd
from lib.preprocessing import *
from lib.cleaning import *
import seaborn as sns


# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from mpl_toolkits.basemap import Basemap
# import matplotlib.pyplot as plt
# %matplotlib inline
# import missingno as msno

In [4]:
df = load_sample(percentage=1, csv_file="data/train_full.csv")
df.head(3)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_18765,User_Id_1342,Bike,3,Business,25,4,12:51:20 PM,25,4,...,1:51:28 PM,9,20.2,NaN,-1.255189,36.782203,-1.302583,36.767081,Rider_Id_263,1422
1,Order_No_14038,User_Id_2956,Bike,3,Business,17,5,12:18:55 PM,17,5,...,1:18:22 PM,17,NaN,NaN,-1.316554,36.700067,-1.258414,36.804800,Rider_Id_205,1893
2,Order_No_8616,User_Id_1787,Bike,3,Business,19,5,3:39:45 PM,19,5,...,4:13:15 PM,12,NaN,NaN,-1.252796,36.800313,-1.228340,36.882276,Rider_Id_697,1363


In [11]:
def get_minute_from_dt_series(series: pd.Series) -> pd.Series:
        return pd.to_datetime(series).dt.hour * 60 + pd.to_datetime(series).dt.minute

other_series = get_minute_from_dt_series(df["Placement - Time"])
temp_control = pd.concat([
    df[[
        "Placement - Day of Month", 
        "Placement - Weekday (Mo = 1)", 
        "Pickup Long", 
        "Pickup Lat",
        "Destination Long",
        "Destination Lat",
        "Distance (KM)",
        "Time from Pickup to Arrival",
        "Temperature"
    ]],
    other_series
], axis=1)

In [12]:
temp_control.head(3)

,Placement - Day of Month,Placement - Weekday (Mo = 1),Pickup Long,Pickup Lat,Destination Long,Destination Lat,Distance (KM),Time from Pickup to Arrival,Temperature,Placement - Time
0,25,4,36.782203,-1.255189,36.767081,-1.302583,9,1422,20.2,771
1,17,5,36.700067,-1.316554,36.804800,-1.258414,17,1893,NaN,738
2,19,5,36.800313,-1.252796,36.882276,-1.228340,12,1363,NaN,939


In [13]:
df.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_18765,User_Id_1342,Bike,3,Business,25,4,12:51:20 PM,25,4,...,1:51:28 PM,9,20.2,NaN,-1.255189,36.782203,-1.302583,36.767081,Rider_Id_263,1422
1,Order_No_14038,User_Id_2956,Bike,3,Business,17,5,12:18:55 PM,17,5,...,1:18:22 PM,17,NaN,NaN,-1.316554,36.700067,-1.258414,36.804800,Rider_Id_205,1893
2,Order_No_8616,User_Id_1787,Bike,3,Business,19,5,3:39:45 PM,19,5,...,4:13:15 PM,12,NaN,NaN,-1.252796,36.800313,-1.228340,36.882276,Rider_Id_697,1363
3,Order_No_11986,User_Id_2216,Bike,3,Business,15,3,9:17:49 AM,15,3,...,10:40:31 AM,9,16.4,NaN,-1.255189,36.782203,-1.294656,36.790832,Rider_Id_933,4142
4,Order_No_24392,User_Id_2593,Bike,3,Business,7,2,10:57:21 AM,7,2,...,11:11:26 AM,1,23.3,NaN,-1.261589,36.792873,-1.255189,36.782203,Rider_Id_449,389


In [2]:
df = load_sample(percentage=1, csv_file="data/train_full.csv")
df = drops(df)
df = impute_temperature(df)
df = combine_weekdays(df)
df.head(3)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but IterativeImputer was fitted without feature names
  warnings.warn(


,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,...,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month
0,Order_No_10084,User_Id_393,Bike,3,Business,12,2,8:53:35 AM,8:53:43 AM,8:54:07 AM,...,14,533.0,-1.316711,36.830156,-1.306378,36.751984,Rider_Id_523,1872,2,12
1,Order_No_564,User_Id_3017,Bike,3,Business,13,5,7:05:02 PM,7:05:35 PM,7:14:42 PM,...,3,1145.0,-1.299313,36.783149,-1.289426,36.762028,Rider_Id_648,797,5,13
2,Order_No_26992,User_Id_1329,Bike,3,Business,24,5,2:10:49 PM,2:12:09 PM,2:23:41 PM,...,3,850.0,-1.273056,36.811298,-1.265595,36.791546,Rider_Id_152,630,5,24


In [22]:
df.loc[(df['Placement - Day of Month'] < df["Fulfillment - Day of Month"])]

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,...,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month
7714,Order_No_3530,User_Id_2457,Bike,3,Business,11,2,742,1:27:30 PM,1:32:38 PM,...,8,22.5,-1.291548,36.829752,-1.255189,36.782203,Rider_Id_247,1854,4,13
18398,Order_No_2800,User_Id_1402,Bike,3,Business,17,1,874,1:16:53 PM,1:59:45 PM,...,11,25.6,-1.225322,36.808550,-1.281754,36.832423,Rider_Id_928,2640,2,18


In [3]:
save_data(df, sql_db="data/cleaned_nelson.db")

In [6]:
raise Exception()

Exception: 

In [7]:
cols = pd.Series(df.columns.format())

metrics = ["Temperature", "Precipitation in millimeters", "Distance (KM)"]  # 3 of these

ids = pd.concat([cols.loc[cols.str.match(".*Id.*")], cols.loc[cols.str.match(".*No.*")]]) # 3 of these

location = ["Pickup Lat", "Pickup Long", "Destination Lat", "Destination Long"] # 4 of these
times = cols.loc[cols.str.match(".*Time.*")] # 6(actually 4) of these

types = ["Vehicle Type", "Platform Type", "Personal or Business", ] # 3 of these


In [8]:
times

7                Placement - Time
8             Confirmation - Time
9        Arrival at Pickup - Time
10                  Pickup - Time
18    Time from Pickup to Arrival
dtype: object

In [9]:
ids

1      User Id
17    Rider Id
0     Order No
dtype: object

In [10]:
desc = pd.read_csv("variable_definitions.csv")
desc

,Variable Name,Description
0,Order No,Unique number identifying the order
1,User Id,Unique number identifying the customer on a pl...
2,Vehicle Type,"For this competition limited to bikes, however..."
3,Platform Type,"Platform used to place the order, there are 4 ..."
4,Personal or Business,Customer type
5,Placement - Day of Month,Placement - Day of Month i.e 1-31
6,Placement - Weekday (Mo = 1),Placement - Weekday (Monday = 1)
7,Placement - Time,Placement - Time - Time of day the order was p...
8,Confirmation - Day of Month,Confirmation - Day of Month i.e 1-31
9,Confirmation - Weekday (Mo = 1),Confirmation - Weekday (Monday = 1)


In [4]:
em = load_sample(percentage=1, sql_db="data/cleaned_emily.db")

In [5]:
em = em.drop(columns=["level_0", "index"]).rename({
    "Order_No" : "order_no",
    "User_Id" : "user_id",
    "Rider_Id" : "rider_id"
})

In [6]:
em.head(3)

,Order_No,User_Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,...,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider_Id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month
0,27800,2330,Bike,3,Business,10,5,9:27:52 AM,9:29:17 AM,9:42:47 AM,...,14,567.0,-1.292564,36.897645,-1.379808,36.929053,690,1829,5,10
1,24479,2869,Bike,3,Business,5,2,8:57:04 AM,8:57:31 AM,9:32:19 AM,...,10,537.0,-1.258414,36.804800,-1.302583,36.767081,880,1096,2,5
2,3218,1642,Bike,3,Business,24,3,2:14:44 PM,2:15:01 PM,2:23:34 PM,...,12,854.0,-1.230821,36.867904,-1.272807,36.796236,96,1897,3,24


In [7]:
save_data(em, sql_db="data/cleaned_emily_nelson.db")

In [32]:
riders_clean = load_sample(percentage=1, sql_db="data/cleaned_Alex.db")
# riders_clean.rename({"id" : "Rider_Id"}, axis=1, inplace=True)
riders_clean.drop(columns="index", inplace=True)
riders_clean["id"] = riders_clean["id"].astype(int)
riders_clean.set_index("id", inplace=True)
riders_clean.head(3)
# riders_clean.dtypes

,orders,age,average_rating,number_rating
id,,,,
630,2588,1285,13.3,576
599,3117,2255,13.7,878
148,4671,941,14.1,973


In [19]:
other_clean = load_sample(percentage=1, sql_db="data/cleaned_emily_nelson.db")
other_clean.head(3)

,index,Order_No,User_Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,...,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider_Id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month
0,3605,28157,2045,Bike,3,Personal,5,5,10:33:27 AM,10:33:44 AM,...,26,633.0,-1.327234,36.711050,-1.224831,36.801257,789,3546,5,5
1,7943,6877,2069,Bike,3,Business,7,3,4:03:29 PM,4:03:45 PM,...,17,963.0,-1.331619,36.847976,-1.228911,36.881876,699,2157,3,7
2,7808,23242,1329,Bike,3,Business,26,2,6:30:58 PM,6:31:40 PM,...,6,1110.0,-1.273056,36.811298,-1.288212,36.778334,907,1271,2,26


In [43]:
cleaned_all = other_clean.join(riders_clean, on="Rider_Id", rsuffix="_rider").drop(columns="index")

In [44]:
save_data(cleaned_all, sql_db="data/cleaned_alex_emily_nelson.db")